<a href="https://colab.research.google.com/github/igors96/SVR_Bagging_Concreto/blob/main/concreto_bagging_svr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
arquivo = pd.read_csv('/content/Concrete Compressive Strength.csv')
arquivo.head()

,Cement (component 1)(kg in a m^3 mixture),Blast Furnace Slag (component 2)(kg in a m^3 mixture),Fly Ash (component 3)(kg in a m^3 mixture),Water (component 4)(kg in a m^3 mixture),Superplasticizer (component 5)(kg in a m^3 mixture),Coarse Aggregate (component 6)(kg in a m^3 mixture),Fine Aggregate (component 7)(kg in a m^3 mixture),Age (day),"Concrete compressive strength(MPa, megapascals)"
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.986111
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.887366
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.269535
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.052780
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.296075


In [2]:
faltantes = arquivo.isnull().sum()
faltantes_percentual = (faltantes/len(arquivo['Age (day)']))*100
print(faltantes_percentual)

Cement (component 1)(kg in a m^3 mixture)                0.0
Blast Furnace Slag (component 2)(kg in a m^3 mixture)    0.0
Fly Ash (component 3)(kg in a m^3 mixture)               0.0
Water  (component 4)(kg in a m^3 mixture)                0.0
Superplasticizer (component 5)(kg in a m^3 mixture)      0.0
Coarse Aggregate  (component 6)(kg in a m^3 mixture)     0.0
Fine Aggregate (component 7)(kg in a m^3 mixture)        0.0
Age (day)                                                0.0
Concrete compressive strength(MPa, megapascals)          0.0
dtype: float64


In [3]:
x = arquivo.drop('Concrete compressive strength(MPa, megapascals) ', axis = 1)
y = arquivo['Concrete compressive strength(MPa, megapascals) ']

In [4]:
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

modelo = BaggingRegressor(n_estimators = 100, max_samples = 0.5, n_jobs = -1)
kfold = KFold(n_splits = 3, shuffle = True)
resultado = cross_val_score(modelo,x,y,cv = kfold, n_jobs = -1)
print(resultado.mean())

0.8862231753116937


In [5]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

modelo1 = BaggingRegressor(base_estimator = GradientBoostingRegressor(n_estimators = 100), n_estimators = 100, max_samples = 0.5, n_jobs = -1)
kfold1 = KFold(n_splits = 3, shuffle = True)
resultado1 = cross_val_score(modelo1,x,y,cv = kfold1, n_jobs = -1)
print(resultado1.mean())

0.8869736845059829


In [6]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

modelo2 = BaggingRegressor(base_estimator = GradientBoostingRegressor(n_estimators = 100), n_estimators = 100, max_samples = 0.5, n_jobs = -1)
kfold2 = KFold(n_splits = 3, shuffle = True)
resultado2 = cross_val_score(modelo2,x,y,cv = kfold2, n_jobs = -1, scoring = 'neg_mean_absolute_error')
print(resultado2.mean())

-4.083218244349219


In [7]:
from sklearn.svm import SVR
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler

normalizador = MinMaxScaler(feature_range = (0,1))
x_norm = normalizador.fit_transform(x)

modelo3 = SVR()
kfold3 = KFold(n_splits = 3, shuffle = True)
resultado3 = cross_val_score(modelo3,x_norm,y,cv = kfold3, n_jobs = -1)
print(resultado3.mean())

0.5720078559810652


In [8]:
import numpy as np
from sklearn.model_selection import GridSearchCV

c = np.array([1.0,0.95,1.05,1.1,1.2,2,0.9,0.8])
kernel = ['linear', 'poly', 'rbf', 'sigmoid']
polinomio = np.array([2,3,4])
epsilon = np.array([0.1,0.2,0.05])
valores_grid = {'C': c, 'kernel': kernel, 'degree': polinomio, 'epsilon': epsilon}

modelo4 = SVR()

kfold = KFold(n_splits = 3, shuffle = True)
gridSVM = GridSearchCV(estimator = modelo4, param_grid = valores_grid, cv = kfold, n_jobs = -1)
gridSVM.fit(x_norm, y)

print('Melhor valor constante: ', gridSVM.best_estimator_.C)
print('Melhor kernel: ', gridSVM.best_estimator_.kernel)
print('Melhor grau polinômio: ', gridSVM.best_estimator_.degree)
print('Melhor epsilon: ', gridSVM.best_estimator_.epsilon)
print('R2: ', gridSVM.best_score_)

Melhor valor constante:  2.0
Melhor kernel:  poly
Melhor grau polinômio:  4
Melhor epsilon:  0.2
R2:  0.8126688339007119
